In [1]:
!python -V

Python 3.9.13


In [ ]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

In [ ]:
#!pip install pyarrow fastparquet

In [2]:
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [53]:
df = pd.read_parquet('./yellow_tripdata_2022-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


### Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data

In [54]:
len(df.columns)

19

### Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [56]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

In [57]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df.duration = df.duration.apply(lambda duration: duration.total_seconds()/60)

In [58]:
round(df['duration'].std(),2)

46.45

### Question 3: Fraction of the records left after dropping the outliers

In [59]:
df["duration"].describe(percentiles=[0.9, 0.93, 0.95, 0.98, 0.99]).apply(lambda x: format(x, 'f'))

count    2463931.000000
mean          14.212203
std           46.445305
min        -3442.400000
50%           10.183333
90%           24.633333
93%           28.300000
95%           31.933333
98%           42.150000
99%           50.850000
max         8513.183333
Name: duration, dtype: object

In [60]:
df["duration"].std()

46.44530513776499

In [61]:
stats.iqr(df["duration"])

9.850000000000001

In [62]:
df["duration"].mean()

14.212202918831741

In [63]:
upper = 1.5 * stats.iqr(df["duration"]) + df["duration"].quantile(0.75)
upper

30.94166666666667

In [64]:
df["duration"].isna().sum()

0

In [65]:
round((1 - (df[df["duration"] >= upper].count().iloc[0] / df["duration"].count())),2)

0.95

### Question 4: Dimensionality after OHE

In [66]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [67]:
df.iloc[:, :-1].shape[1]

19

In [68]:
pd.get_dummies(df).shape[1]

21

In [69]:
pd.get_dummies(df).shape[1] - df.iloc[:, :-1].shape[1]

2

### Question 5: RMSE on train

### Question 6: RMSE on validation

### myTEST

In [70]:
((df["duration"] >= 1) & (df["duration"] <= 60)).mean()

0.9827547930522406

In [71]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [72]:
#df = df.dropna(inplace=True)

In [73]:
df.isna().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
duration                     0
dtype: int64

In [97]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance", "total_amount"]

In [98]:
df[categorical] = df[categorical].astype(str)

In [99]:
df[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [100]:
df[numerical].dtypes

trip_distance    float64
total_amount     float64
dtype: object

In [101]:
dv = DictVectorizer()

In [102]:
train_data = df[categorical + numerical].dropna().to_dict(orient='records')

In [103]:
X_train = dv.fit_transform(train_data)

In [104]:
X_train

<2463931x520 sparse matrix of type '<class 'numpy.float64'>'
	with 9855724 stored elements in Compressed Sparse Row format>

In [105]:
y_train = df["duration"].values

In [106]:
y_train

array([17.81666667,  8.4       ,  8.96666667, ..., 11.        ,
       12.05      , 27.        ])

In [107]:
lr = LinearRegression()

In [108]:
lr.fit(X_train, y_train)

LinearRegression()

In [109]:
y_pred = lr.predict(X_train)

In [110]:
y_pred

array([10.88980323, 17.67442775, 15.81251291, ..., 11.6188376 ,
       12.42377237, 33.8320479 ])

In [111]:
model_df = pd.DataFrame(data={"train": y_train, "pred": y_pred})

In [112]:
model_df.head()

,train,pred
0,17.816667,10.889803
1,8.400000,17.674428
2,8.966667,15.812513
3,10.033333,11.429759
4,37.533333,12.944294


In [113]:
rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

46.03547733387083

### Validation Data

In [116]:
df_valid = pd.read_parquet('yellow_tripdata_2022-02.parquet')

len(df_valid.columns)

19

In [117]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979431 entries, 0 to 2979430
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [118]:
df_valid['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df_valid['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

df_valid['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df_valid['duration'] = df_valid['duration'].apply(lambda duration: duration.total_seconds()/60)


In [119]:
round(df_valid['duration'].std(),2)

46.45

In [121]:
df_valid[categorical] = df_valid[categorical].astype(str)

dv = DictVectorizer()

valid_data = df_valid[categorical + numerical].dropna().to_dict(orient='records')



In [127]:
X_valid = dv.fit_transform(valid_data)
y_valid = df_valid["duration"].values

In [128]:
lr = LinearRegression()

In [129]:
lr.fit(X_valid, y_valid)

ValueError: Found input variables with inconsistent numbers of samples: [2979431, 2463931]